# Building a ReAct Agent from Scratch with OpenAI

**Author**: Sergio Masa Avis  
**Date**: October 2025  
**Goal**: Understand and implement the ReAct pattern by building a simple AI agent that can reason and take actions.

---

## Table of Contents

1. [What is the ReAct Pattern?](#1-what-is-the-react-pattern)
2. [Architecture Overview](#2-architecture-overview)
3. [Implementation](#3-implementation)
   - [Setup & Basic Agent Class](#31-setup--environment)
   - [ReAct System Prompt](#33-the-react-system-prompt)
   - [Tools Implementation](#34-tools-implementation)
   - [Action Parsing & Execution](#35-action-parsing--execution)
4. [Examples & Usage](#4-examples--usage)
5. [Limitations & Considerations](#5-limitations--considerations)
6. [Next Steps](#6-next-steps)

---

## 1. What is the ReAct Pattern?

**ReAct** = **Rea**soning + **Act**ing

ReAct is a framework where an AI agent alternates between **thinking** (reasoning) and **doing** (acting) to solve tasks. Instead of trying to answer a question in one shot, the agent:

1. **Thinks** about what it needs to do
2. **Acts** by using a tool or taking an action
3. **Observes** the result of that action
4. **Repeats** this cycle until it has enough information to answer

### The ReAct Loop

```mermaid
graph LR
    A[🤔 Thought] --> B[⚡ Action]
    B --> C[👁️ Observation]
    C --> A
    
    style A fill:#e1f5ff,stroke:#01579b,stroke-width:2px
    style B fill:#fff3e0,stroke:#e65100,stroke-width:2px
    style C fill:#f1f8e9,stroke:#33691e,stroke-width:2px
```

**The ReAct Workflow (Cyclical)**

### Why is ReAct Important?

**Problem with direct prompting**: LLMs don't have access to real-time information or external tools. If you ask "What's the price of 2 apples and 3 bananas?", the model:
- Doesn't know current prices
- Can't do reliable math
- Might hallucinate an answer

**Solution with ReAct**: The agent can:
- **Reason**: "I need to get the price of apples and bananas"
- **Act**: Call `get_fruit_price` tool
- **Observe**: Receive actual prices
- **Reason**: "Now I can calculate the total"
- **Act**: Call `calculate_total_price` tool
- **Answer**: Return accurate result

### Single Iteration vs Multi-Iteration

**Single Iteration (Simple Task)**

```mermaid
graph LR
    U[👤 User<br/>Request] --> T[💭 Thought]
    T --> A[⚡ Action]
    A --> O[👁️ Observation]
    O --> R[✅ Result]
    
    style U fill:#e3f2fd,stroke:#1565c0
    style T fill:#f3e5f5,stroke:#6a1b9a
    style A fill:#fff3e0,stroke:#e65100
    style O fill:#e8f5e9,stroke:#2e7d32
    style R fill:#fce4ec,stroke:#c2185b
```

**Multi-Iteration (Complex Task)**

```mermaid
graph TD
    U[👤 User Request] --> T1[💭 Thought 1]
    T1 --> A1[⚡ Action 1]
    A1 --> O1[👁️ Observation 1]
    O1 --> T2[💭 Thought 2]
    T2 --> A2[⚡ Action 2]
    A2 --> O2[👁️ Observation 2]
    O2 --> T3[💭 Thought 3]
    T3 --> A3[⚡ Action 3]
    A3 --> O3[👁️ Observation 3]
    O3 --> Answer[💡 Answer]
    
    style U fill:#e3f2fd,stroke:#1565c0
    style Answer fill:#c8e6c9,stroke:#2e7d32,stroke-width:3px
```

**Note**: In this notebook, we'll implement a **single-iteration** ReAct agent for simplicity. Multi-iteration requires a loop that we'll explore in future notebooks.

---

## 2. Architecture Overview

Our ReAct agent consists of 4 main components:

```mermaid
graph TD
    subgraph System["🤖 ReAct Agent System"]
        A["1️⃣ Agent Class<br/>━━━━━━━━━<br/>• Conversation history<br/>• OpenAI communication<br/>• Response handling"]
        B["2️⃣ System Prompt<br/>━━━━━━━━━<br/>• ReAct instructions<br/>• Available tools<br/>• Examples"]
        C["3️⃣ Tools<br/>━━━━━━━━━<br/>• get_fruit_price()<br/>• calculate_total_price()"]
        D["4️⃣ Parser & Executor<br/>━━━━━━━━━<br/>• Parse actions<br/>• Call functions<br/>• Return observations"]
    end
    
    style A fill:#e1f5ff,stroke:#01579b
    style B fill:#fff3e0,stroke:#e65100
    style C fill:#f3e5f5,stroke:#6a1b9a
    style D fill:#e8f5e9,stroke:#2e7d32
```

### Data Flow

```mermaid
sequenceDiagram
    participant U as 👤 User
    participant A as 🤖 Agent
    participant P as 🔍 Parser
    participant T as 🛠️ Tools
    
    U->>A: Question: "What is the price of 2 apples?"
    A->>A: Generate response with ReAct prompt
    A->>P: "Thought: I need price first<br/>Action: get_fruit_price: apple<br/>PAUSE"
    P->>P: Parse with regex<br/>Extract: action="get_fruit_price"<br/>input="apple"
    P->>T: Call get_fruit_price("apple")
    T->>P: "The price of an apple is $1.5"
    P->>U: Return observation
```

### Key Design Decisions

| Decision | Rationale |
|----------|----------|
| 🗂️ **Stateful Agent** | Maintains conversation history for context |
| 📝 **Explicit System Prompt** | Defines ReAct behavior through examples |
| 🔤 **String-based Parsing** | Simple but brittle (regex matching) |
| 1️⃣ **Single-shot Execution** | One action per query (no loops yet) |

---

## 3. Implementation

Let's build our ReAct agent step by step.

---

### 3.1 Setup & Environment

First, we need to:
1. Load OpenAI API key from environment
2. Test that the connection works
3. Initialize the OpenAI client

**Why this matters**: You need a valid API key to use OpenAI models. We use `python-dotenv` to load it from a `.env` file for security (never hardcode API keys!).

In [ ]:
import os
import re

from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables from .env file
_ = load_dotenv()

# Verify API key is present
assert os.getenv("OPENAI_API_KEY"), "Missing OPENAI_API_KEY environment variable"

# Initialize OpenAI client
client = OpenAI()

# Quick test to verify connection
resp = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say 'Hello, world!' in Spanish if you are ready."}],
)

print("✅ OpenAI client initialized successfully!")
print(f"Response: {resp.choices[0].message.content}")

### 3.2 The Agent Class

This is the core of our system. The `Agent` class:
- Maintains conversation history
- Manages the system prompt
- Handles communication with OpenAI API

#### Design Highlights

**`__init__` method**: Sets up the agent with an optional system message. The system message defines the agent's behavior.

**`__call__` method**: Makes the agent callable like a function. When you do `agent("question")`, it:
1. Adds user message to history
2. Sends to OpenAI
3. Gets response
4. Adds response to history
5. Returns the response

**`execute` method**: Does the actual API call. Sends the **entire conversation history** to maintain context.

#### Why Conversation History?

LLMs are **stateless** - they don't remember previous messages unless you include them. By maintaining `self.messages`, we give the LLM context:

```
┌──────────────────────────┬────────────────────────────────────┐
│   Without History ❌     │      With History ✅               │
├──────────────────────────┼────────────────────────────────────┤
│ User: "What's 2+2?"      │ [System: "You are a calculator"]   │
│ LLM: "4"                 │ [User: "What's 2+2?"]              │
│                          │ [Assistant: "4"]                   │
│ User: "Add 3"            │ [User: "Add 3"]                    │
│ LLM: "3? To what?" ❌    │ [Assistant: "7"] ✅                │
└──────────────────────────┴────────────────────────────────────┘
```

In [ ]:
class Agent:
    """
    A simple conversational AI agent that maintains chat history.
    
    The agent wraps OpenAI's chat completion API and manages:
    - System prompts that define behavior
    - Conversation history for context
    - Stateful interactions across multiple turns
    """
    
    def __init__(self, system: str = ""):
        """
        Initialize the agent with an optional system message.
        
        Args:
            system (str): System prompt that defines the agent's behavior.
                         For ReAct, this will contain the Thought-Action-Observation
                         instructions and available tools.
        """
        self.client = OpenAI()
        self.system = system
        self.messages = []  # Conversation history
        
        # If system message provided, add it as the first message
        if self.system:
            self.messages.append({"role": "system", "content": self.system})
    
    def __call__(self, message: str) -> str:
        """
        Make the agent callable. This allows: agent("your question")
        
        Args:
            message (str): User's input message
            
        Returns:
            str: Agent's response
        """
        # Add user message to history
        self.messages.append({"role": "user", "content": message})
        
        # Get response from LLM
        result = self.execute()
        
        # Add assistant response to history
        self.messages.append({"role": "assistant", "content": result})
        
        return result
    
    def execute(self) -> str:
        """
        Execute the conversation by calling OpenAI API.
        
        Sends the ENTIRE conversation history to maintain context.
        Uses temperature=0 for deterministic outputs (important for action parsing).
        
        Returns:
            str: The model's response content
        """
        completion = self.client.chat.completions.create(
            model="gpt-4o-mini",  # Cheaper, faster model (good for prototyping)
            temperature=0,         # Deterministic outputs (critical for parsing)
            messages=self.messages # Full conversation history
        )
        
        return completion.choices[0].message.content

print("✅ Agent class defined")

### 3.3 The ReAct System Prompt

This is where the **magic happens**. The system prompt teaches the LLM to:
1. Think step-by-step
2. Use tools when needed
3. Output in a structured format we can parse
4. **Be specific** about what it's reasoning about

#### Anatomy of a Good ReAct Prompt

```mermaid
graph TD
    A["📋 System Prompt"] --> B["1️⃣ Loop Definition<br/>Define Thought-Action-Observation cycle"]
    A --> C["2️⃣ Component Definitions<br/>Explain what each part means"]
    A --> D["3️⃣ Available Tools<br/>List tools with syntax examples"]
    A --> E["4️⃣ Example Session<br/>Show complete workflow with SPECIFIC details"]
    
    E --> E1["✨ Key: Examples show<br/>SPECIFIC items being processed"]
    
    style A fill:#e1f5ff,stroke:#01579b,stroke-width:3px
    style E1 fill:#fff3e0,stroke:#e65100,stroke-width:2px
```

#### Why This Structure?

**"PAUSE" keyword**: Signals to our parser that the LLM is waiting for an observation. Without it, the LLM might continue generating and hallucinate the observation.

**Specific example session**: LLMs learn best from examples (few-shot learning). The example shows:
- ✅ **SPECIFIC items**: "I need to find out the price of an apple" (not just "I need to get the price")
- ✅ How to format actions
- ✅ When to pause
- ✅ How observations look
- ✅ When to give final answer

**Tool syntax specification**: Being explicit about format (e.g., `apple: 2, banana: 3`) reduces parsing errors.

In [ ]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

Use Thought to describe your thoughts about the question you have been asked.
IMPORTANT: In your Thought, always explicitly mention the SPECIFIC items, parameters, or values you are working with.

Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_total_price:
e.g. calculate_total_price: apple: 2, banana: 3
Runs a calculation for the total price based on the quantity and prices of the fruits

get_fruit_price:
e.g. get_fruit_price: apple
returns the price of the fruit when given its name.

Example session:

Question: What is the total price for 2 apples and 3 bananas?
Thought: I need to find out the price of an apple first, then the price of a banana, and finally calculate the total for 2 apples and 3 bananas.
Action: get_fruit_price: apple
PAUSE

Observation: The price of an apple is $1.5

Thought: Now I know an apple costs $1.5. I need to get the price of a banana next.
Action: get_fruit_price: banana
PAUSE

Observation: The price of a banana is $1.2

Thought: Now I have both prices: apple is $1.5 and banana is $1.2. I can calculate the total for 2 apples and 3 bananas.
Action: calculate_total_price: apple: 2, banana: 3
PAUSE

Observation: The total price is $6.6

Answer: The total price for 2 apples and 3 bananas is $6.6
""".strip()

print("✅ ReAct system prompt defined")
print(f"\n📊 Prompt length: {len(prompt)} characters")
print(f"\n🔍 Key features:")
print("   • Explicit instruction to be SPECIFIC in Thoughts")
print("   • Detailed example showing multi-step reasoning")
print("   • Each Thought mentions exact items being processed")

### 3.4 Tools Implementation

Tools are the **actions** the agent can take. They're just Python functions that:
- Accept string inputs (from LLM)
- Return string outputs (for LLM to observe)
- Perform actual computations or lookups

#### Tool Design Principles

```mermaid
graph LR
    A["🔤 String Input<br/>from LLM"] --> B["⚙️ Process<br/>(lookup/compute)"] --> C["🔤 String Output<br/>to LLM"]
    
    style A fill:#e3f2fd,stroke:#1565c0
    style B fill:#f3e5f5,stroke:#6a1b9a
    style C fill:#e8f5e9,stroke:#2e7d32
```

| Principle | Why |
|-----------|-----|
| 🔤 **String in, String out** | LLMs work with text |
| 📝 **Descriptive outputs** | Full sentences help LLM understand |
| 🛡️ **Error handling** | Graceful failures with helpful messages |
| 📋 **Logging** | Debug what tools are called |

#### Our Fruit Price Tools

```mermaid
graph TD
    subgraph T1["🛠️ get_fruit_price"]
        I1["📥 Input: 'apple'"] --> L1["🔍 Lookup: fruit_prices['apple']"] --> O1["📤 Output: 'The price of an apple is $1.5'"]
    end
    
    subgraph T2["🛠️ calculate_total_price"]
        I2["📥 Input: 'apple: 2, banana: 3'"] --> P2["🔧 Parse: [(apple, 2), (banana, 3)]"]
        P2 --> C2["🧮 Calculate: 1.5*2 + 1.2*3 = 6.6"]
        C2 --> O2["📤 Output: 'The total price is $6.6'"]
    end
    
    style T1 fill:#e1f5ff,stroke:#01579b
    style T2 fill:#fff3e0,stroke:#e65100
```

In [ ]:
# Our "database" of fruit prices
fruit_prices = {
    "apple": 1.5,
    "banana": 1.2,
    "orange": 1.3,
    "grapes": 2.0,
}

def get_fruit_price(fruit: str) -> str:
    """
    Tool: Look up the price of a single fruit.
    
    Args:
        fruit (str): Name of the fruit (e.g., "apple")
        
    Returns:
        str: Human-readable price string or error message
        
    Example:
        >>> get_fruit_price("apple")
        "The price of an apple is $1.5"
    """
    print(f"🔍 [get_fruit_price] Looking up price for: {fruit}")
    
    # Normalize input (handle case variations)
    fruit = fruit.lower().strip()
    
    if fruit in fruit_prices:
        return f"The price of an {fruit} is ${fruit_prices[fruit]}"
    else:
        return f"Sorry, I don't have the price for {fruit}. Available fruits: {', '.join(fruit_prices.keys())}"

def calculate_total_price(fruits: str) -> str:
    """
    Tool: Calculate total price for multiple fruits with quantities.
    
    Args:
        fruits (str): Comma-separated list in format "fruit: quantity, fruit: quantity"
                     Example: "apple: 2, banana: 3"
        
    Returns:
        str: Total price string or error message
        
    Example:
        >>> calculate_total_price("apple: 2, banana: 3")
        "The total price is $6.6"
    """
    print(f"🧮 [calculate_total_price] Calculating total for: {fruits}")
    
    total = 0.0
    fruit_list = fruits.split(', ')  # Split by comma
    
    for item in fruit_list:
        try:
            # Parse "fruit: quantity" format
            fruit, quantity = item.split(': ')
            fruit = fruit.lower().strip()
            quantity = int(quantity)
            
            # Check if fruit exists in our database
            if fruit in fruit_prices:
                item_total = fruit_prices[fruit] * quantity
                total += item_total
                print(f"  ├─ {quantity}x {fruit} @ ${fruit_prices[fruit]} = ${item_total}")
            else:
                return f"Sorry, I don't have the price for {fruit}"
                
        except ValueError as e:
            return f"Error parsing input '{item}'. Expected format: 'fruit: quantity'"
    
    print(f"  └─ Total: ${total}")
    return f"The total price is ${total}"

# Map tool names to functions (for easy lookup)
known_actions = {
    "get_fruit_price": get_fruit_price,
    "calculate_total_price": calculate_total_price,
}

print("✅ Tools defined")
print(f"🛠️  Available tools: {list(known_actions.keys())}")

### 3.5 Action Parsing & Execution

This is the **glue** that connects the LLM's text output to our Python functions.

#### The Parsing Challenge

```mermaid
graph LR
    A["📄 LLM Output<br/>'Thought: ...<br/>Action: get_fruit_price: apple<br/>PAUSE'"] --> B["🔍 Regex Parse<br/>Extract tool & input"]
    B --> C["✅ Validation<br/>Tool exists?"]
    C --> D["⚡ Execute<br/>Call function"]
    D --> E["👁️ Observation<br/>Return result"]
    
    style A fill:#e3f2fd,stroke:#1565c0
    style B fill:#f3e5f5,stroke:#6a1b9a
    style C fill:#fff3e0,stroke:#e65100
    style D fill:#e8f5e9,stroke:#2e7d32
    style E fill:#fce4ec,stroke:#c2185b
```

#### Regex Pattern Breakdown

```python
action_re = re.compile(r'^Action: (\w+): (.*)$')
```

| Part | Meaning |
|------|--------|
| `^Action:` | Line must start with "Action:" |
| `(\w+)` | 📦 **Group 1**: Tool name (letters, numbers, underscore) |
| `:` | Literal colon separator |
| `(.*)$` | 📦 **Group 2**: Everything until end of line (input) |

**Example match**:
```
Input:   "Action: get_fruit_price: apple"
         ─────── ─────────────── ─────
         prefix     Group 1     Group 2
                    
Result:  action = "get_fruit_price"
         input  = "apple"
```

In [ ]:
# Regex to parse action lines from LLM output
# Format: "Action: tool_name: input"
action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question: str, verbose: bool = True):
    """
    Execute a query using the ReAct agent pattern.
    
    This function:
    1. Creates an agent with the ReAct system prompt
    2. Sends the user question to the agent
    3. Parses the agent's response for actions
    4. Executes the first action found
    5. Returns the observation
    
    Args:
        question (str): The user's question
        verbose (bool): If True, print detailed execution info
        
    Returns:
        str: The observation from executing the action, or None if no action found
        
    Example:
        >>> query("What is the price of an apple?")
        # Prints agent reasoning and returns: "The price of an apple is $1.5"
    """
    # Step 1: Create agent with ReAct prompt
    bot = Agent(system=prompt)
    
    # Step 2: Get initial response from LLM
    result = bot(question)  # Uses __call__ method
    
    if verbose:
        print("\n" + "="*70)
        print("📋 AGENT RESPONSE")
        print("="*70)
        print(result)
        print("="*70 + "\n")
    
    # Step 3: Parse the response for actions
    # Split by newlines and check each line for action pattern
    actions = [
        action_re.match(line)
        for line in result.split('\n')
        if action_re.match(line)
    ]
    
    # Step 4: Execute the first action found
    if actions:
        # Extract tool name and input from regex groups
        action, action_input = actions[0].groups()
        
        # Validate that the tool exists
        if action not in known_actions:
            raise ValueError(
                f"Unknown action: {action}\n"
                f"Available actions: {list(known_actions.keys())}"
            )
        
        if verbose:
            print(f"⚡ EXECUTING ACTION")
            print(f"   Tool: {action}")
            print(f"   Input: {action_input}")
            print()
        
        # Call the tool function
        observation = known_actions[action](action_input)
        
        if verbose:
            print()
            print("👁️  OBSERVATION")
            print(f"   {observation}")
            print()
        
        return observation
    else:
        if verbose:
            print("ℹ️  No action found in response (agent may have answered directly)")
        return None

print("✅ Query function defined")

## 4. Examples & Usage

Let's test our ReAct agent with different scenarios.

---

### Example 1: Simple Price Lookup

**Scenario**: User asks for the price of a single fruit.

**Expected behavior**:
1. Agent thinks: "I need to find out the price of **an orange**" (specific!)
2. Agent acts: Calls `get_fruit_price`
3. Tool returns: Price information

**What to observe**:
- ✅ The "Thought" shows reasoning with SPECIFIC fruit name
- ✅ The "Action" is properly formatted
- ✅ "PAUSE" indicates waiting for observation
- ✅ Tool is called correctly

In [ ]:
query("What is the price of an orange?")

### Example 2: Unknown Fruit

**Scenario**: User asks about a fruit not in our database.

**Expected behavior**:
- Agent mentions "dragonfruit" specifically in its Thought
- Agent still tries to call the tool
- Tool returns helpful error message
- Lists available fruits

In [ ]:
query("What is the price of a dragonfruit?")

### Example 3: Multiple Fruits (Single Iteration Limitation)

**Scenario**: User asks for total price of multiple fruits.

**Expected behavior**:
- Agent realizes it needs to get individual prices first
- Agent's Thought mentions "2 apples and 3 bananas" specifically
- Makes ONE tool call (first action)
- ❌ **Stops** (doesn't continue to next action)

**This demonstrates the limitation of single-iteration execution**

In [ ]:
query("What is the total price for 2 apples and 3 bananas?")

## 5. Limitations & Considerations

This implementation is **educational**, not production-ready. Here are its limitations:

---

### 🚫 Limitation 1: Single Iteration Only

**Problem**: Agent executes ONE action then stops.

```mermaid
graph TD
    Q["❓ User: Total for 2 apples + 3 bananas?"] --> T1["💭 Thought 1: Need apple price"]
    T1 --> A1["⚡ Action: get_fruit_price: apple"]
    A1 --> O1["👁️ Observation: $1.5"]
    O1 --> STOP["🛑 STOPS HERE<br/>(Should continue but doesn't)"]
    
    O1 -.-> T2["💭 Thought 2: Need banana price"]
    T2 -.-> A2["⚡ Action: get_fruit_price: banana"]
    A2 -.-> O2["👁️ Observation: $1.2"]
    O2 -.-> T3["💭 Thought 3: Calculate total"]
    T3 -.-> A3["⚡ Action: calculate_total"]
    A3 -.-> Answer["💡 Answer: $6.6"]
    
    style STOP fill:#ffebee,stroke:#c62828,stroke-width:3px
    style T2 stroke-dasharray: 5 5
    style A2 stroke-dasharray: 5 5
    style O2 stroke-dasharray: 5 5
    style T3 stroke-dasharray: 5 5
    style A3 stroke-dasharray: 5 5
    style Answer stroke-dasharray: 5 5,fill:#e8f5e9,stroke:#2e7d32
```

**Fix**: Implement a loop (covered in next notebook)

---

### 🚫 Limitation 2: Brittle String Parsing

**Problem**: We use regex to parse actions. This breaks if LLM's format varies slightly.

| Format | Works? |
|--------|--------|
| `"Action: get_fruit_price: apple"` | ✅ |
| `"Action: get_fruit_price : apple"` | ❌ (extra space) |
| `"Action:get_fruit_price: apple"` | ❌ (no space) |
| `"action: get_fruit_price: apple"` | ❌ (lowercase) |
| `"ACTION: get_fruit_price: apple"` | ❌ (uppercase) |

**Better solution**: Use **function calling** (structured outputs) where OpenAI returns JSON.

---

### 🚫 Limitation 3-8: Other Important Issues

| # | Limitation | Impact | Solution |
|---|------------|--------|----------|
| 3 | No error recovery | Can't retry or use alternatives | Add loop + fallback tools |
| 4 | No output validation | Tools can crash ungracefully | Try-catch + type checking |
| 5 | Hardcoded tools | Code changes for new tools | Dynamic tool registry |
| 6 | No memory | Forgets across sessions | Add persistence layer |
| 7 | No cost tracking | Unknown API spending | Track tokens + costs |
| 8 | Weak input validation | Tools crash on bad input | Robust parsing |

### 📊 Production Checklist

```
Current          Production
─────────────    ──────────────────────
❌ Single iter   ✅ Multi-turn loop
❌ Regex         ✅ Function calling
❌ No recovery   ✅ Retry logic
❌ No validation ✅ Type checking
❌ Hardcoded     ✅ Dynamic registry
❌ No memory     ✅ Persistence
❌ No tracking   ✅ Cost monitoring
❌ Weak parsing  ✅ Robust validation
```

---

## 6. Next Steps

### 🎯 Immediate Exercises

1. **Add a loop** for multi-turn conversations
2. **Create a new tool** (e.g., `get_discount` for bulk purchases)
3. **Implement usage tracking**
4. **Add structured logging**

### 📚 Learning Path

```mermaid
graph LR
    A["✅ 01: Basic ReAct<br/>(this notebook)"] --> B["📝 02: Multi-iteration<br/>(with loops)"]
    B --> C["🔧 03: Function Calling<br/>(structured outputs)"]
    C --> D["🌐 04: LangGraph<br/>(production framework)"]
    
    style A fill:#c8e6c9,stroke:#2e7d32,stroke-width:3px
```

### 🔗 Resources

**Papers**:
- [ReAct: Synergizing Reasoning and Acting](https://arxiv.org/abs/2210.03629)
- [Toolformer: Language Models Can Teach Themselves to Use Tools](https://arxiv.org/abs/2302.04761)

**Documentation**:
- [OpenAI Function Calling](https://platform.openai.com/docs/guides/function-calling)
- [LangGraph Docs](https://langchain-ai.github.io/langgraph/)

---

## 🎓 Key Takeaways

```
┌──────────────────────────────────────────────────────┐
│  5 Things to Remember                                │
├──────────────────────────────────────────────────────┤
│  1. ReAct = Reasoning + Acting (cyclical loop)       │
│  2. System prompt defines behavior (with examples)   │
│  3. Tools extend LLM to external world               │
│  4. Specificity in prompts improves outputs          │
│  5. Simple implementations teach fundamentals        │
└──────────────────────────────────────────────────────┘
```

---

## 📝 Conclusion

You've built a working ReAct agent from scratch! You now understand:

- ✅ How the ReAct pattern works
- ✅ Why agents need tools
- ✅ How to parse LLM outputs
- ✅ Why being specific in prompts matters
- ✅ What's needed for production systems

**Happy building! 🚀**

---

*Last updated: October 2025*  
*For questions: sergio.masavi@gmail.com*